# PJM Hourly Energy Consumption Case

PJM Interconnection LLC (PJM) is a regional transmission organization (RTO) in the United States. It is part of the Eastern Interconnection grid operating an electric transmission system serving all or parts of Delaware, Illinois, Indiana, Kentucky, Maryland, Michigan, New Jersey, North Carolina, Ohio, Pennsylvania, Tennessee, Virginia, West Virginia, and the District of Columbia.

The hourly power consumption data comes from PJM's website and are in megawatts (MW).

### LSTM Training Step - By Sabrina Otoni da Silva - 2024/04

In [1]:
from pathlib import Path 

import numpy as np
import pandas as pd

# import matplotlib.pyplot as plt
# import plotly.express as px
# import plotly.graph_objs as go

from sklearn.preprocessing import MinMaxScaler
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, classification_report, ConfusionMatrixDisplay, accuracy_score, confusion_matrix

# import tensorflow as tf
# import tensorflow.keras.layers as L
# from tensorflow.keras import optimizers, Sequential, Model
# from keras.callbacks import EarlyStopping, LearningRateScheduler

# import pickle

In [ ]:
datapath = Path('../data/d02_intermediate')

In [ ]:
df = pd.read_csv(f'{datapath}/pjme_train.csv')
df.set_index('datetime', inplace=True)
df.index = pd.to_datetime(df.index)
df.sort_index(inplace=True)
df.dropna(axis=0, how='any', inplace=True)

In [ ]:
def temporalize(X, y, lookback):
    '''
    To convert input data into 3-D
    array as required for LSTM network.
    '''
    output_X = []
    output_y = []
    for i in range(len(X)-lookback-1):
        t = []
        for j in range(1,lookback+1):
            t.append(X[[(i+j+1)], :])
        output_X.append(t)
        output_y.append(y[i+lookback+1])
        
    return output_X, output_y

In [ ]:
n_features = df.shape[1] - 1
timesteps = 24

X_train, y_train = temporalize(X = np.array(df[['hour', 'dayofweek', 'quarter', 'month', 'year', 'dayofyear', 'day', 'weekofyear', 'lag1', 'lag2', 'lag3']]), 
                   y = np.array(df[['pjme_mw']]), 
                   lookback = timesteps)

X_train = np.array(X_train)
X_train = X_train.reshape(X_train.shape[0], timesteps, n_features)

In [ ]:
scalerX = MinMaxScaler(feature_range=(0, 1))
scalerX.fit(X_train)
X_train= scalerX.transform(X_train)
X_train.shape
# scalerY = MinMaxScaler(feature_range=(0, 1))
# scalerY.fit(y_train)
# y_train = scalerY.transform(y_train).reshape(-1, y_train.shape[1], 1)

In [ ]:
# lstm_model = Sequential()
# lstm_model.add(L.LSTM(100, activation='relu', input_shape=(X_train.shape[1], 1)))
# lstm_model.add(L.RepeatVector(X_train.shape[1]))
# lstm_model.add(L.LSTM(100, activation='relu', return_sequences=True))
# lstm_model.add(L.TimeDistributed(L.Dense(1)))
# lstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
# serie_size =  X_train.shape[1]
# n_features =  X_train.shape[2]

# encoder_decoder = Sequential()
# encoder_decoder.add(L.LSTM(serie_size, activation='relu', input_shape=(serie_size, n_features), return_sequences=True))
# encoder_decoder.add(L.LSTM(6, activation='relu', return_sequences=True))
# encoder_decoder.add(L.LSTM(1, activation='relu'))
# encoder_decoder.add(L.RepeatVector(serie_size))
# encoder_decoder.add(L.LSTM(serie_size, activation='relu', return_sequences=True))
# encoder_decoder.add(L.LSTM(6, activation='relu', return_sequences=True))
# encoder_decoder.add(L.TimeDistributed(L.Dense(1)))
# encoder_decoder.summary()

In [ ]:
# encoder_decoder.compile(loss='mse', optimizer=optimizers.Adam(0.0001))

# encoder_decoder_history = encoder_decoder.fit(X_train, X_train, 
#                                               batch_size=128, 
#                                               epochs=20, 
#                                               verbose=2)

In [ ]:
# rpt_vector_layer = Model(inputs=encoder_decoder.inputs, outputs=encoder_decoder.layers[3].output)
# time_dist_layer = Model(inputs=encoder_decoder.inputs, outputs=encoder_decoder.layers[5].output)
# encoder_decoder.layers

In [ ]:
# encoder = Model(inputs=encoder_decoder.inputs, outputs=encoder_decoder.layers[2].output)

In [ ]:
# train_encoded = encoder.predict(X_train)
# print('Encoded time-series shape', train_encoded.shape)
# print('Encoded time-series sample', train_encoded[0])

In [ ]:
# df['encoded'] = train_encoded
# df['label'] = y_train

# df.head(10)

In [ ]:
# es = EarlyStopping(patience=10, verbose=0, min_delta=0.001, monitor='loss', mode='auto', restore_best_weights=True)
# red_lr = LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)

# hist = lstm_model.fit(
#     x=X_train,
#     y=y_train,
#     batch_size=100,
#     shuffle=False,
#     epochs=300,
#     verbose=1,
#     callbacks=[es, red_lr]
# )